# Pipeline d'entraînement des modèles NER avec spaCy (Google colab / GPU)

Referencias:
*   NER4ARCHIVES : https://github.com/NER4Archives-project/spacy_ner_training_pipeline
*   Spacy transformers : https://www.youtube.com/watch?v=Y_N_AO39rRg
*   Spacy tranformers : https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-huggingface-hub
!pip install spacy-transformers
!python -m spacy download es_dep_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 56.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires f

Vérifier Spacy en version 3.x et l'utilisation du GPU

In [ ]:
import spacy
import torch
torch.cuda.empty_cache()

print('GPU State : ', spacy.prefer_gpu()) # doit renvoyer True
try:
  print('GPU required : ', spacy.require_gpu(0)) # doit renvoyer True
except ValueError:
  print("No GPU devices detected")
print('Cuda available : ', torch.cuda.is_available()) # doit renvoyer True
print('SPACY VERSION : ', spacy.__version__)

GPU State :  True
GPU required :  True
Cuda available :  True
SPACY VERSION :  3.7.2


In [ ]:
!python -m spacy info

2024-01-31 08:57:09.970990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 08:57:09.971046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 08:57:09.972208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 08:57:11.223117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded perfor

**Modelo 3 categorias genericas (Per Loc Org)**

In [ ]:
!cat "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catplo_TestDS_A_project-6-at-2024-01-30-19-18-f0cdbb83_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catplo_TestDS_B_project-7-at-2024-01-30-19-19-09667f06_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catplo_TestDS_C_project-8-at-2024-01-30-19-20-f341919e_modif.conll" > "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif_ALL.conll"

In [ ]:
#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif_ALL.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catEvalDS_project-3-at-2023-12-15-12-38-8e39bcf8_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

2024-01-30 20:51:01.689822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:51:01.689888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:51:01.691537: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 20:51:02.799991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (664 documents): /content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/bin_containers/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d088

In [ ]:
#!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catTrainDS_project-3-at-2023-12-14-23-16-3e7d0880_modif_ALL.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catEvalDS_project-3-at-2023-12-15-12-38-8e39bcf8_modif.spacy" ./dev.spacy

In [ ]:
#!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/base_config_es_trf.cfg" ./config.cfg
!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/config_gpu_trf_es_config.cfg" ./config.cfg


2024-01-30 20:55:44.259051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:55:44.259121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:55:44.260503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 20:55:45.380513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy debug config config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

2024-01-30 20:55:59.022434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:55:59.022488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:55:59.023673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 20:56:00.106253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ 

In [ ]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

2024-01-30 20:56:10.587757: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:56:10.587818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:56:10.588969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 20:56:11.733848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================ Data file validation ============================
tokenizer_config.json: 100% 364/364 [00:00<00:00, 1.92MB/s]
config.json: 100% 648/648 [00:00<00:00, 3.35MB/s]
vocab.txt: 100% 242k/242k [00:00<00:00, 1.84MB/s]
t

In [ ]:
!python -m spacy train --gpu-id 0 config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es_ALL/"

2024-01-30 20:57:25.060372: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 20:57:25.060427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 20:57:25.061761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 20:57:26.129142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: /content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es_ALL
ℹ Saving to output directory: /content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_

Salida entrenamiento

```
============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        1258.55    872.89    0.88    0.68    1.23    0.01
  9     200      103858.38  29793.53   90.93   89.13   92.80    0.91
 19     400         375.18    367.61   91.33   89.53   93.21    0.91
 29     600          44.16     59.43   92.06   91.13   93.00    0.92
 39     800          10.29     15.38   92.21   90.66   93.83    0.92
 49    1000          39.67     66.05   94.25   95.75   92.80    0.94
 59    1200          12.30     25.89   92.32   91.85   92.80    0.92
 69    1400          21.85     14.25   92.78   92.98   92.59    0.93
 79    1600         200.92    133.24   91.95   91.11   92.80    0.92
 90    1800          31.33     39.29   92.41   94.81   90.12    0.92
 99    2000          26.89     15.85   92.09   91.99   92.18    0.92
109    2200          12.24     13.40   93.03   94.11   91.98    0.93
119    2400          33.32     43.14   93.14   92.67   93.62    0.93
130    2600          44.37     41.14   91.95   91.11   92.80    0.92
✔ Saved pipeline to output directory
/content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es/model-last
```



**Modelo 3 categorias Loc (asentamientos, naturel, territorios)**

In [ ]:
!cat "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_A_project-10-at-2024-01-30-23-14-e7d97da2_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_A_project-10-at-2024-01-30-23-14-e7d97da2_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_B_project-11-at-2024-01-30-23-18-00c140c4_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_C_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll" > "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll"

In [ ]:
#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

#!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/3catLoc_evalDS_project-4-at-2023-12-26-09-37-1e54cf98_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

2024-01-31 09:02:56.972871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:02:56.972931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:02:56.974471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 09:02:58.138433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (701 documents): /content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/bin_containers/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23

In [ ]:
#!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catLoc_trainDS_project-4-at-2023-12-26-09-57-88cd17ba_modif.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/LocCat_testDS_ALL_ABC_project-12-at-2024-01-30-23-20-d7dc64a1_modif.spacy" ./train.spacy

!cp "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/3catLoc_evalDS_project-4-at-2023-12-26-09-37-1e54cf98_modif.spacy" ./dev.spacy

In [ ]:
#!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/base_config_es_trf.cfg" ./config.cfg
!python -m spacy init fill-config "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/config/config_gpu_trf_es_config.cfg" ./config.cfg

2024-01-31 09:03:57.608262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:03:57.608316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:03:57.609536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 09:03:58.674532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy debug config config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

2024-01-31 09:04:11.998477: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:04:11.998529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:04:11.999697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 09:04:13.055671: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ 

In [ ]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

2024-01-31 09:04:25.086286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:04:25.086342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:04:25.087555: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 09:04:26.174666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================ Data file validation ============================
tokenizer_config.json: 100% 364/364 [00:00<00:00, 1.47MB/s]
config.json: 100% 648/648 [00:00<00:00, 2.97MB/s]
vocab.txt: 100% 242k/242k [00:00<00:00, 9.30MB/s]
t

In [ ]:
!python -m spacy train --gpu-id 0 config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es_drp05/"

2024-01-31 09:05:06.243003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 09:05:06.243053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 09:05:06.244689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 09:05:07.709597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es_drp05
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of Be

Salida entrenamiento:

```
============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        1239.89   1801.06    4.61    2.47   34.18    0.05
  9     200      121020.48  58058.61   74.32   72.01   76.79    0.74
 19     400         584.95    643.10   74.38   71.90   77.04    0.74
 29     600         162.81    164.75   78.52   76.08   81.12    0.79
 39     800          16.03     12.45   76.08   73.98   78.32    0.76
 49    1000          37.71     51.41   75.51   74.75   76.28    0.76
 59    1200          81.94     95.95   78.93   78.54   79.34    0.79
 69    1400         120.78    100.42   73.48   72.75   74.23    0.73
 79    1600         234.76    145.42   77.83   75.24   80.61    0.78
 89    1800          44.38     53.82   76.14   73.86   78.57    0.76
100    2000          14.46      8.88   75.52   72.92   78.32    0.76
110    2200           2.47      2.09   77.00   74.35   79.85    0.77
120    2400           5.43      5.26   76.66   73.93   79.59    0.77
130    2600          15.36     10.53   73.96   71.33   76.79    0.74
140    2800          67.46     78.62   78.73   75.59   82.14    0.79
✔ Saved pipeline to output directory
/content/drive/MyDrive/Colab
Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es/model-last
```



Las etapas siguientes estan aqui:
*   Evaluacion : https://colab.research.google.com/drive/1xmu8x46s7Zt9hXBbEx3kMYMdh8ST3PB2#scrollTo=-TgbH_G-MYB_


